In [160]:
import pandas as pd
import numpy as np


from sklearn.linear_model import Perceptron
from lib_adult import accuracy
from lib_adult import metric
from lib_adult.resample import split_stratified_train_test 

from lib_adult import transform
from importlib import reload
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
 
#"error", "ignore", "always", "default", "module" or "once"

In [161]:
reload(resample)
reload(transform)

<module 'lib_adult.transform' from 'C:\\Users\\s_raf\\Documents\\2019.1\\Machine Learning\\Pre\\lib_adult\\transform.py'>

# Carregamento dos dados

In [162]:
columns = ['Age','Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status',
           'Occupation','Relationship', 'Race', 'Sex', 
           'Capital-gain', 'Capital-loss', 'Hours-per-week','Native-country', 'Label']
original_data = pd.read_csv('adult.data', na_values='?', sep= ',\s+', names=columns, engine='python')

# Pre-processamento

In [163]:
original_data = original_data.drop(['Education'], axis=1)
# Indentificação das colunas com dados categóricos
cat_columns  = original_data.select_dtypes(['object']).columns
#'Object' é sobre as colunas categóricas


In [164]:
original_data.head()

,Age,Workclass,Fnlwgt,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Label
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [165]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
Age               32561 non-null int64
Workclass         30725 non-null object
Fnlwgt            32561 non-null int64
Education-num     32561 non-null int64
Marital-status    32561 non-null object
Occupation        30718 non-null object
Relationship      32561 non-null object
Race              32561 non-null object
Sex               32561 non-null object
Capital-gain      32561 non-null int64
Capital-loss      32561 non-null int64
Hours-per-week    32561 non-null int64
Native-country    31978 non-null object
Label             32561 non-null object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB


In [166]:
original_data.isna().sum()

Age                  0
Workclass         1836
Fnlwgt               0
Education-num        0
Marital-status       0
Occupation        1843
Relationship         0
Race                 0
Sex                  0
Capital-gain         0
Capital-loss         0
Hours-per-week       0
Native-country     583
Label                0
dtype: int64

In [167]:
# Tratamento de colunas com missing values. Substituição pela moda (apenas colunas categóricas)
#for column in columns:
#    if ' ?' in original_data[column].unique():
#        
#        original_data[column] = original_data[column].replace(' ?',original_data[column].mode()[0])
#[0] está pegando os valores

In [168]:
for column in ['Workclass', 'Occupation', 'Native-country']:
    original_data[column] = original_data[column].replace(np.nan, original_data[column].mode()[0])
    

In [169]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
Age               32561 non-null int64
Workclass         32561 non-null object
Fnlwgt            32561 non-null int64
Education-num     32561 non-null int64
Marital-status    32561 non-null object
Occupation        32561 non-null object
Relationship      32561 non-null object
Race              32561 non-null object
Sex               32561 non-null object
Capital-gain      32561 non-null int64
Capital-loss      32561 non-null int64
Hours-per-week    32561 non-null int64
Native-country    32561 non-null object
Label             32561 non-null object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB


In [170]:
# Transformar categóricos em one-hot encoding
original_data = pd.get_dummies(original_data, columns=cat_columns, prefix=cat_columns, drop_first=True)
original_data = original_data.rename(columns={'Label_>50K':'Label'})

In [171]:
original_data.head()

,Age,Fnlwgt,Education-num,Capital-gain,Capital-loss,Hours-per-week,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,Workclass_Self-emp-inc,...,Native-country_Puerto-Rico,Native-country_Scotland,Native-country_South,Native-country_Taiwan,Native-country_Thailand,Native-country_Trinadad&Tobago,Native-country_United-States,Native-country_Vietnam,Native-country_Yugoslavia,Label
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38,215646,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,234721,7,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,28,338409,13,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
original_data.columns

Index(['Age', 'Fnlwgt', 'Education-num', 'Capital-gain', 'Capital-loss',
       'Hours-per-week', 'Workclass_Local-gov', 'Workclass_Never-worked',
       'Workclass_Private', 'Workclass_Self-emp-inc',
       'Workclass_Self-emp-not-inc', 'Workclass_State-gov',
       'Workclass_Without-pay', 'Marital-status_Married-AF-spouse',
       'Marital-status_Married-civ-spouse',
       'Marital-status_Married-spouse-absent', 'Marital-status_Never-married',
       'Marital-status_Separated', 'Marital-status_Widowed',
       'Occupation_Armed-Forces', 'Occupation_Craft-repair',
       'Occupation_Exec-managerial', 'Occupation_Farming-fishing',
       'Occupation_Handlers-cleaners', 'Occupation_Machine-op-inspct',
       'Occupation_Other-service', 'Occupation_Priv-house-serv',
       'Occupation_Prof-specialty', 'Occupation_Protective-serv',
       'Occupation_Sales', 'Occupation_Tech-support',
       'Occupation_Transport-moving', 'Relationship_Not-in-family',
       'Relationship_Other-relative

# Separação 

##### Features e Labels

In [173]:
X = original_data.drop(['Label'], axis=1)
y = original_data[['Label']]

##### Dados de treino e teste

In [175]:
perc_train          = 0.7
seed                = 42
#idx: índices que retornam na função de split_stratified_train_test no módulo

idx_train, idx_test = split_stratified_train_test(y, perc_train, seed)

In [177]:
#.loc: Acessar um grupo de linhas e colunas por labels ou uma matriz booleana.
X_train = X.loc[idx_train]
X_test = X.loc[idx_test]

y_train = y.loc[idx_train]
y_test = y.loc[idx_test]

In [186]:
np.unique(y_train, return_counts=True)

(array([0, 1], dtype=uint8), array([17329,  5464], dtype=int64))

In [188]:
17329/(17329+5464), 5464/(17329+5464)

(0.760277278111701, 0.23972272188829905)

In [190]:
np.unique(y_test, return_counts=True)

(array([0, 1], dtype=uint8), array([7390, 2377], dtype=int64))

In [192]:
7390/(7390+2377), 2377/(7390+2377)

(0.7566294665711067, 0.2433705334288932)

In [193]:
np.unique(y, return_counts=True)

(array([0, 1], dtype=uint8), array([24720,  7841], dtype=int64))

In [195]:
24720/(24720+7841), 7841/(24720+7841)

(0.7591904425539756, 0.2408095574460244)

In [196]:
X_train

,Age,Fnlwgt,Education-num,Capital-gain,Capital-loss,Hours-per-week,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,Workclass_Self-emp-inc,...,Native-country_Portugal,Native-country_Puerto-Rico,Native-country_Scotland,Native-country_South,Native-country_Taiwan,Native-country_Thailand,Native-country_Trinadad&Tobago,Native-country_United-States,Native-country_Vietnam,Native-country_Yugoslavia
14160,27,160178,10,0,0,38,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
27048,45,50567,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
28868,29,185908,13,0,0,55,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5667,30,190040,13,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
7827,29,189346,10,2202,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15382,51,108435,14,0,0,47,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4641,58,93664,9,15024,0,60,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8943,22,148431,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
216,50,313321,12,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
5121,50,71417,9,3103,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


# Normalização dos Dados

In [203]:
X_train.max()

Age                                               90
Fnlwgt                                       1484705
Education-num                                     16
Capital-gain                                   99999
Capital-loss                                    4356
Hours-per-week                                    99
Workclass_Local-gov                                1
Workclass_Never-worked                             1
Workclass_Private                                  1
Workclass_Self-emp-inc                             1
Workclass_Self-emp-not-inc                         1
Workclass_State-gov                                1
Workclass_Without-pay                              1
Marital-status_Married-AF-spouse                   1
Marital-status_Married-civ-spouse                  1
Marital-status_Married-spouse-absent               1
Marital-status_Never-married                       1
Marital-status_Separated                           1
Marital-status_Widowed                        

In [205]:
X_train.min()

Age                                             17
Fnlwgt                                       12285
Education-num                                    1
Capital-gain                                     0
Capital-loss                                     0
Hours-per-week                                   1
Workclass_Local-gov                              0
Workclass_Never-worked                           0
Workclass_Private                                0
Workclass_Self-emp-inc                           0
Workclass_Self-emp-not-inc                       0
Workclass_State-gov                              0
Workclass_Without-pay                            0
Marital-status_Married-AF-spouse                 0
Marital-status_Married-civ-spouse                0
Marital-status_Married-spouse-absent             0
Marital-status_Never-married                     0
Marital-status_Separated                         0
Marital-status_Widowed                           0
Occupation_Armed-Forces        

In [210]:
X_train.mean()

Age                                              38.598385
Fnlwgt                                       189515.925416
Education-num                                    10.061028
Capital-gain                                   1068.373097
Capital-loss                                     83.259597
Hours-per-week                                   40.399026
Workclass_Local-gov                               0.064274
Workclass_Never-worked                            0.000175
Workclass_Private                                 0.755583
Workclass_Self-emp-inc                            0.034440
Workclass_Self-emp-not-inc                        0.076164
Workclass_State-gov                               0.039530
Workclass_Without-pay                             0.000483
Marital-status_Married-AF-spouse                  0.000614
Marital-status_Married-civ-spouse                 0.459176
Marital-status_Married-spouse-absent              0.013250
Marital-status_Never-married                      0.3269

In [202]:
(X_train - X_train.min())/ (X_train.max() - X_train.min())

,Age,Fnlwgt,Education-num,Capital-gain,Capital-loss,Hours-per-week,Workclass_Local-gov,Workclass_Never-worked,Workclass_Private,Workclass_Self-emp-inc,...,Native-country_Portugal,Native-country_Puerto-Rico,Native-country_Scotland,Native-country_South,Native-country_Taiwan,Native-country_Thailand,Native-country_Trinadad&Tobago,Native-country_United-States,Native-country_Vietnam,Native-country_Yugoslavia
14160,0.136986,0.100442,0.600000,0.000000,0.000000,0.377551,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
27048,0.383562,0.025999,0.533333,0.000000,0.000000,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28868,0.164384,0.117917,0.800000,0.000000,0.000000,0.551020,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5667,0.178082,0.120723,0.800000,0.000000,0.000000,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7827,0.164384,0.120252,0.600000,0.022020,0.000000,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15382,0.465753,0.065301,0.866667,0.000000,0.000000,0.469388,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4641,0.561644,0.055269,0.533333,0.150242,0.000000,0.602041,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8943,0.068493,0.092464,0.533333,0.000000,0.000000,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
216,0.452055,0.204450,0.733333,0.000000,0.000000,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5121,0.452055,0.040160,0.533333,0.031030,0.000000,0.397959,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [199]:
transformer = transform.Normalize()
transformer.fit(X_train)

X_train_normalized = transformer.transform(X_train)
X_test_normalized = transformer.transform(X_test)

# Método de Classificação

##### Perceptron

In [243]:
# Utilizando o Perceptron
clf = Perceptron()
clf.fit(X_train_normalized, y_train)
y_pred = clf.predict(X_test_normalized)
acc_perceptron = accuracy(y_test, y_pred)
acc_perceptron

C:\Users\s_raf\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\s_raf\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8245111088358759

In [245]:
# Utilizando o Perceptron
clf = Perceptron()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc_perceptron = accuracy(y_test, y_pred)
acc_perceptron

C:\Users\s_raf\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\s_raf\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7772089689771681

#### KNN

In [249]:
knn = KNeighborsClassifier()
knn.fit(X_train_normalized, y_train)
y_pred = knn.predict(X_test_normalized)
acc_knn = accuracy(y_test, y_pred)
acc_knn

C:\Users\s_raf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


0.8248182655882051

##### Decision Tree

In [250]:
dt = DecisionTreeClassifier()
dt.fit(X_train_normalized, y_train)
y_hat = knn.predict(X_test_normalized)
acc_dt = accuracy(y_test, y_hat)
acc_dt

0.8248182655882051